# EX05 - Exercícios simples e avançados com PySpark
~120M obs. de desempenho de vôos (1987 -2008)


In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 40.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=788aedc5d07153853b2ae51a45aa89e5f8409eec8023c747a857ec38df46d6d4
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


## Obtenção dos arquivos com os dados

Vamos baixar os arquivos no formato `.parquet` (arquivos PARQUET - formato colunar de rápida leitura, sem parsing de campos). **Atenção**: caso a totalidade dos arquivos seja copiada (de 1987 até 2008), o conteúdo total perfaz a quantia de ~1.5GBytes.

In [2]:
%%bash
for i in $(seq 1987 1988); do # altere aqui a configuração para baixar outros anos
  FILENAME=${i}.parquet
  wget -q https://www.inf.ufrgs.br/~schnorr/cd009/ad08/${FILENAME} -O ${FILENAME}
done

Baixar metadados

In [3]:
%%bash
for file in airports.csv carriers.csv plane-data.csv; do
   echo
   echo $file
   echo
   wget -q https://www.inf.ufrgs.br/~schnorr/cd009/ad08/${file} -O ${file}
   #cat ${file}
done


airports.csv


carriers.csv


plane-data.csv



## Iniciar a sessão local Spark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

## Leitura dos arquivos

Podemos empregar wildcards para ler vários arquivos.

In [5]:
#spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
df = spark.read.parquet('*.parquet')
df.count()

6513922

In [6]:
type(df)

pyspark.sql.dataframe.DataFrame

Podemos ver os tipos das colunas e o nome das colunas:

In [7]:
df.columns

['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'TailNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Origin',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay']

Ver o esquema das colunas (tipos).

In [8]:
df.printSchema()

root
 |-- Year: long (nullable = true)
 |-- Month: long (nullable = true)
 |-- DayofMonth: long (nullable = true)
 |-- DayOfWeek: long (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: long (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- CRSArrTime: long (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: long (nullable = true)
 |-- TailNum: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- CRSElapsedTime: long (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- Cancelled: long (nullable = true)
 |-- CancellationCode: double (nullable = true)
 |-- Diverted: long (nullable = true)
 |-- CarrierDelay: double (nullable = 

Selecionar algumas colunas e visualizar os primeiros valores.

In [9]:
df \
  .select("Year", "Month", "UniqueCarrier", "DepTime") \
  .sample(True, 10/df.count(), seed=1) \
  .collect()

[Row(Year=1988, Month=5, UniqueCarrier='DL', DepTime=2100.0),
 Row(Year=1988, Month=7, UniqueCarrier='TW', DepTime=1225.0),
 Row(Year=1988, Month=7, UniqueCarrier='HP', DepTime=2255.0),
 Row(Year=1988, Month=7, UniqueCarrier='PI', DepTime=713.0),
 Row(Year=1988, Month=8, UniqueCarrier='PA (1)', DepTime=1325.0),
 Row(Year=1988, Month=10, UniqueCarrier='UA', DepTime=1106.0),
 Row(Year=1988, Month=10, UniqueCarrier='DL', DepTime=1912.0),
 Row(Year=1988, Month=12, UniqueCarrier='US', DepTime=2042.0),
 Row(Year=1988, Month=12, UniqueCarrier='UA', DepTime=843.0),
 Row(Year=1988, Month=12, UniqueCarrier='HP', DepTime=1343.0),
 Row(Year=1987, Month=10, UniqueCarrier='PI', DepTime=1910.0),
 Row(Year=1987, Month=10, UniqueCarrier='PI', DepTime=702.0)]

# Operações comuns

Vamos ver algumas operações comuns.

Selecionar os valores únicos da coluna `Year`.

In [10]:
df.select("Year").distinct().collect()

[Row(Year=1988), Row(Year=1987)]

Selecionar algumas colunas.

In [11]:
df \
  .select("Origin", "Dest", "DepDelay") \
  .sample(True, 10/df.count(), seed=0) \
  .collect()

[Row(Origin='ORD', Dest='CLE', DepDelay=0.0),
 Row(Origin='LGA', Dest='MSP', DepDelay=0.0),
 Row(Origin='MEM', Dest='LIT', DepDelay=0.0),
 Row(Origin='CLT', Dest='ATL', DepDelay=9.0),
 Row(Origin='RDU', Dest='IAD', DepDelay=-4.0),
 Row(Origin='LGA', Dest='BOS', DepDelay=None),
 Row(Origin='BWI', Dest='CLT', DepDelay=0.0),
 Row(Origin='MDT', Dest='ORD', DepDelay=4.0),
 Row(Origin='IAH', Dest='EWR', DepDelay=35.0)]

Como remover algumas colunas

In [12]:
df2 = df.drop("Year", "Month", "DayofMonth")
df2.printSchema()

root
 |-- DayOfWeek: long (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: long (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- CRSArrTime: long (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: long (nullable = true)
 |-- TailNum: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- CRSElapsedTime: long (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- Cancelled: long (nullable = true)
 |-- CancellationCode: double (nullable = true)
 |-- Diverted: long (nullable = true)
 |-- CarrierDelay: double (nullable = true)
 |-- WeatherDelay: double (nullable = true)
 |-- NASDelay: double (nullable = true)
 |-- SecurityDelay:

Vamos alterar uma coluna:

In [13]:
df \
  .withColumn("novacoluna", df.DayOfWeek - df.DayofMonth) \
  .select("Origin", "Dest", "DepDelay", "DayofMonth", "Month") \
  .sample(True, 10/df.count(), seed=0) \
  .collect()

[Row(Origin='ORD', Dest='CLE', DepDelay=0.0, DayofMonth=19, Month=2),
 Row(Origin='LGA', Dest='MSP', DepDelay=0.0, DayofMonth=21, Month=3),
 Row(Origin='MEM', Dest='LIT', DepDelay=0.0, DayofMonth=21, Month=4),
 Row(Origin='CLT', Dest='ATL', DepDelay=9.0, DayofMonth=11, Month=7),
 Row(Origin='RDU', Dest='IAD', DepDelay=-4.0, DayofMonth=8, Month=8),
 Row(Origin='LGA', Dest='BOS', DepDelay=None, DayofMonth=12, Month=8),
 Row(Origin='BWI', Dest='CLT', DepDelay=0.0, DayofMonth=6, Month=12),
 Row(Origin='MDT', Dest='ORD', DepDelay=4.0, DayofMonth=26, Month=12),
 Row(Origin='IAH', Dest='EWR', DepDelay=35.0, DayofMonth=26, Month=12)]

Podemos filtrar as linhas baseado em condições

In [14]:
df \
  .filter((df.DayofMonth == 7) & (df.Origin == "SFO")) \
  .select("Dest", "DepDelay", "DayOfWeek", "DayofMonth") \
  .sample(True, 0.1, seed=0) \
  .head(10)

[Row(Dest='BUR', DepDelay=53.0, DayOfWeek=4, DayofMonth=7),
 Row(Dest='LGB', DepDelay=-1.0, DayOfWeek=4, DayofMonth=7),
 Row(Dest='SAN', DepDelay=74.0, DayOfWeek=4, DayofMonth=7),
 Row(Dest='SAN', DepDelay=17.0, DayOfWeek=4, DayofMonth=7),
 Row(Dest='LGB', DepDelay=2.0, DayOfWeek=4, DayofMonth=7),
 Row(Dest='SAN', DepDelay=-1.0, DayOfWeek=4, DayofMonth=7),
 Row(Dest='LAX', DepDelay=47.0, DayOfWeek=4, DayofMonth=7),
 Row(Dest='ORD', DepDelay=0.0, DayOfWeek=4, DayofMonth=7),
 Row(Dest='ORD', DepDelay=15.0, DayOfWeek=4, DayofMonth=7),
 Row(Dest='LAX', DepDelay=23.0, DayOfWeek=4, DayofMonth=7)]

Podemo pegar o final da tabela:

In [15]:
df \
  .select("Origin", "Dest", "DepDelay") \
  .tail(10)

[Row(Origin='ORD', Dest='EWR', DepDelay=0.0),
 Row(Origin='ORD', Dest='EWR', DepDelay=0.0),
 Row(Origin='ORD', Dest='EWR', DepDelay=0.0),
 Row(Origin='ORD', Dest='EWR', DepDelay=0.0),
 Row(Origin='ORD', Dest='EWR', DepDelay=0.0),
 Row(Origin='ORD', Dest='EWR', DepDelay=0.0),
 Row(Origin='ORD', Dest='EWR', DepDelay=0.0),
 Row(Origin='ORD', Dest='EWR', DepDelay=0.0),
 Row(Origin='BOS', Dest='EWR', DepDelay=0.0),
 Row(Origin='BOS', Dest='EWR', DepDelay=15.0)]

In [16]:
df \
  .select("Origin", "Dest", "DepDelay") \
  .head(10)

[Row(Origin='SYR', Dest='BWI', DepDelay=17.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=3.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=75.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=3.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=10.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=1.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=0.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=-4.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=0.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=18.0)]

Podemos reordenar os dados por colunas.

In [17]:
df \
  .sort("Month") \
  .select("Origin", "Dest", "DepDelay") \
  .head(10)

[Row(Origin='SYR', Dest='BWI', DepDelay=0.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=10.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=-4.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=75.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=3.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=1.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=0.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=17.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=3.0),
 Row(Origin='SYR', Dest='BWI', DepDelay=18.0)]

Ou aindar ordenar de maneira decrescente, por exemplo para mostrar qual foi o vôo com o maior atraso:

In [18]:
df \
  .select("Year", "ArrDelay", "Origin", "Dest") \
  .sort("ArrDelay", ascending=False) \
  .head(10)

[Row(Year=1988, ArrDelay=1394.0, Origin='HDN', Dest='ORD'),
 Row(Year=1988, ArrDelay=1381.0, Origin='DCA', Dest='SDF'),
 Row(Year=1988, ArrDelay=1379.0, Origin='CLT', Dest='TPA'),
 Row(Year=1988, ArrDelay=1179.0, Origin='STX', Dest='EWR'),
 Row(Year=1988, ArrDelay=1169.0, Origin='MIA', Dest='STT'),
 Row(Year=1988, ArrDelay=1167.0, Origin='EWR', Dest='BOS'),
 Row(Year=1988, ArrDelay=1155.0, Origin='STT', Dest='STX'),
 Row(Year=1988, ArrDelay=1114.0, Origin='STT', Dest='MIA'),
 Row(Year=1987, ArrDelay=1033.0, Origin='SFO', Dest='SNA'),
 Row(Year=1988, ArrDelay=1012.0, Origin='DTW', Dest='IND')]

Podemos criar grupos com `groupby`. Por exemplo, vamos agrupar os dados por `Origin` e `Dest`.

In [19]:
df \
  .groupby("Origin", "Dest") \
  .count() \
  .sort("count", ascending=False) \
  .head(10)

[Row(Origin='SFO', Dest='LAX', count=26614),
 Row(Origin='LAX', Dest='SFO', count=26532),
 Row(Origin='LAX', Dest='PHX', count=17268),
 Row(Origin='PHX', Dest='LAX', count=17053),
 Row(Origin='LAX', Dest='LAS', count=15449),
 Row(Origin='LAS', Dest='LAX', count=15135),
 Row(Origin='LGA', Dest='BOS', count=15051),
 Row(Origin='SJC', Dest='LAX', count=14790),
 Row(Origin='LAX', Dest='SJC', count=14594),
 Row(Origin='BOS', Dest='LGA', count=13916)]

Podemos empregar `groupby` e obter qual foi o maior atraso para cada uma das rotas Origem/Destino utilizando a função `max`. Para terminar, ordenamos pelo pior atraso para saber qual o par Origem/Destino que tem o maior atraso.

In [20]:
df \
  .groupby("Origin", "Dest") \
  .max("ArrDelay") \
  .sort("max(ArrDelay)", ascending=False) \
  .limit(10) \
  .collect()

[Row(Origin='HDN', Dest='ORD', max(ArrDelay)=1394.0),
 Row(Origin='DCA', Dest='SDF', max(ArrDelay)=1381.0),
 Row(Origin='CLT', Dest='TPA', max(ArrDelay)=1379.0),
 Row(Origin='STX', Dest='EWR', max(ArrDelay)=1179.0),
 Row(Origin='MIA', Dest='STT', max(ArrDelay)=1169.0),
 Row(Origin='EWR', Dest='BOS', max(ArrDelay)=1167.0),
 Row(Origin='STT', Dest='STX', max(ArrDelay)=1155.0),
 Row(Origin='STT', Dest='MIA', max(ArrDelay)=1114.0),
 Row(Origin='SFO', Dest='SNA', max(ArrDelay)=1033.0),
 Row(Origin='DTW', Dest='IND', max(ArrDelay)=1012.0)]

O uso do encadeamento com o operador `.` e a contrabarra `\` permite calcular de maneira relativamente legível um sumário estatístico dos atrasos, em conjunto com a operação `describe`.

In [21]:
df.groupby("Origin", "Dest") \
  .max("ArrDelay") \
  .sort("max(ArrDelay)", ascending=False) \
  .describe() \
  .show()

+-------+------+----+------------------+
|summary|Origin|Dest|     max(ArrDelay)|
+-------+------+----+------------------+
|  count|  3927|3927|              3894|
|   mean|  null|null|244.94401643554187|
| stddev|  null|null|145.27263738137287|
|    min|   ABE| ABE|             -35.0|
|    max|   YUM| YUM|            1394.0|
+-------+------+----+------------------+



# Exercícios

Carregue a maior quantidade de anos que couber na memória do seu computador.

In [22]:
%%time
df = spark.read.parquet('*.parquet')
dfcarriers = spark.read.csv("carriers.csv", header=True)
dfcarriers.show()

+----+--------------------+
|Code|         Description|
+----+--------------------+
| 02Q|       Titan Airways|
| 04Q|  Tradewind Aviation|
| 05Q| Comlux Aviation, AG|
| 06Q|Master Top Linhas...|
| 07Q| Flair Airlines Ltd.|
| 09Q|      Swift Air, LLC|
| 0BQ|                 DCA|
| 0CQ|ACM AIR CHARTER GmbH|
| 0FQ|Maine Aviation Ai...|
| 0GQ|Inter Island Airw...|
| 0HQ|Polar Airlines de...|
|  0J|          JetClub AG|
| 0JQ|     Vision Airlines|
| 0KQ|Mokulele Flight S...|
| 0LQ|   Metropix UK, LLP.|
| 0MQ|Multi-Aero, Inc. ...|
|  0Q| Flying Service N.V.|
|  16|   PSA Airlines Inc.|
|  17|   Piedmont Airlines|
|  1I|Sky Trek Int'l Ai...|
+----+--------------------+
only showing top 20 rows

CPU times: user 13.5 ms, sys: 2.77 ms, total: 16.2 ms
Wall time: 1.08 s


Responda incluindo blocos de código neste arquivo de análise.

## Simples

Qual a empresa que tem mais vôos não cancelados?

In [23]:
# limit(1) antes do join para limitar a quantidade de dados para fazer o join
df \
  .filter(df.Cancelled == 0) \
  .groupby("UniqueCarrier") \
  .count() \
  .sort("count", ascending=False) \
  .limit(1) \
  .join(dfcarriers, df.UniqueCarrier == dfcarriers.Code, "left") \
  .show()

+-------------+------+----+--------------------+
|UniqueCarrier| count|Code|         Description|
+-------------+------+----+--------------------+
|           DL|933270|  DL|Delta Air Lines Inc.|
+-------------+------+----+--------------------+



Qual a `UniqueCarrier` que tem o maior atraso de chegada (`ArrDelay`)?

In [24]:
df \
  .filter(df.Cancelled == 0) \
  .groupby("UniqueCarrier") \
  .max("ArrDelay") \
  .sort("max(ArrDelay)", ascending=False) \
  .limit(1) \
  .show()  
  

+-------------+-------------+
|UniqueCarrier|max(ArrDelay)|
+-------------+-------------+
|           AA|       1394.0|
+-------------+-------------+



Qual o dia da semana que tem mais vôos?

In [25]:
df \
  .filter(df.Cancelled == 0) \
  .groupby("DayOfWeek") \
  .count() \
  .sort("count", ascending=False) \
  .limit(1) \
  .show() 

+---------+------+
|DayOfWeek| count|
+---------+------+
|        4|942705|
+---------+------+



Qual a empresa que apresentar o maior número de cancelamentos?

In [26]:
df \
  .filter(df.Cancelled == 1) \
  .groupby("UniqueCarrier") \
  .count() \
  .sort("count", ascending=False) \
  .limit(1) \
  .show() 

+-------------+-----+
|UniqueCarrier|count|
+-------------+-----+
|           UA| 9202|
+-------------+-----+



Qual empresa que voou mais milhas?

In [27]:
df \
  .filter(df.Cancelled == 0) \
  .groupby("UniqueCarrier") \
  .sum("Distance") \
  .sort("sum(Distance)", ascending=False) \
  .limit(1) \
  .show() 

+-------------+-------------+
|UniqueCarrier|sum(Distance)|
+-------------+-------------+
|           AA| 6.02741006E8|
+-------------+-------------+



## Avançados

Qual a empresa `UniqueCarrier` que conseguiu recuperar melhor os atrasos durante o vôo?

In [28]:
df \
  .filter(df.Cancelled == 0) \
  .withColumn("Recover", df.DepDelay - df.ArrDelay) \
  .select("UniqueCarrier", "DepDelay", "ArrDelay", "Recover") \
  .groupby("UniqueCarrier") \
  .sum("Recover") \
  .sort("sum(Recover)", ascending=False) \
  .limit(1) \
  .collect()

[Row(UniqueCarrier='CO', sum(Recover)=1355314.0)]

Há um crescimento no número de vôos?

In [29]:
df \
  .filter(df.Cancelled == 0) \
  .groupby("Year") \
  .count() \
  .sort("count", ascending=False) \
  .show() 

+----+-------+
|Year|  count|
+----+-------+
|1988|5151933|
|1987|1292141|
+----+-------+



Qual empresa que apresenta uma redução no número de vôos?

In [30]:
df \
  .filter(df.Cancelled == 0) \
  .groupby(["UniqueCarrier", "Year"]) \
  .count() \
  .sort(["UniqueCarrier", "Year"]) \
  .show() 

+-------------+----+------+
|UniqueCarrier|Year| count|
+-------------+----+------+
|           AA|1987|163165|
|           AA|1988|688146|
|           AS|1987| 21191|
|           AS|1988| 89123|
|           CO|1987|120494|
|           CO|1988|452108|
|           DL|1987|183756|
|           DL|1988|749514|
|           EA|1987|106322|
|           EA|1988|382686|
|           HP|1987| 44863|
|           HP|1988|179898|
|           NW|1987|106686|
|           NW|1988|425266|
|       PA (1)|1987| 16586|
|       PA (1)|1988| 71401|
|           PI|1987|115253|
|           PI|1988|466643|
|           PS|1987| 40827|
|           PS|1988| 41639|
+-------------+----+------+
only showing top 20 rows



Qual a época do ano que apresenta o maior cancelamento de vôos?

In [31]:
df \
  .filter(df.Cancelled == 1) \
  .groupby("Month") \
  .count() \
  .sort("count", ascending=False) \
  .show() 

+-----+-----+
|Month|count|
+-----+-----+
|   12|16122|
|    1|15755|
|   11| 8708|
|    2| 7323|
|   10| 5307|
|    3| 3123|
|    7| 2750|
|    5| 2627|
|    8| 2478|
|    4| 2414|
|    9| 2038|
|    6| 1203|
+-----+-----+



## Livre

Faça uma consulta original para este dataset, potencialmente mais complexa que todas as anteriores.

### Qual tipo de Delay ocasiona um maior total de atraso por mês do ano?

Conhecendo as variáveis: CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay.

In [32]:

df \
  .select("CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay") \
  .summary().show()
  

+-------+------------+------------+--------+-------------+-----------------+
|summary|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-------+------------+------------+--------+-------------+-----------------+
|  count|           0|           0|       0|            0|                0|
|   mean|        null|        null|    null|         null|             null|
| stddev|        null|        null|    null|         null|             null|
|    min|        null|        null|    null|         null|             null|
|    25%|        null|        null|    null|         null|             null|
|    50%|        null|        null|    null|         null|             null|
|    75%|        null|        null|    null|         null|             null|
|    max|        null|        null|    null|         null|             null|
+-------+------------+------------+--------+-------------+-----------------+



Percebemos que nos anos de 1987 e 1988 não foram registrados valores para essas variáveis. Procuramos em anos mais recentes.

Carregando os anos de 2006 e 2007

In [33]:

%%bash
for i in $(seq 2006 2007); do # altere aqui a configuração para baixar outros anos
  FILENAME=${i}.parquet
  wget -q https://www.inf.ufrgs.br/~schnorr/cd009/ad08/${FILENAME} -O ${FILENAME}
done

In [34]:
df_novo = spark.read.parquet('200*.parquet')

In [35]:
df_novo \
  .select("CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay") \
  .summary().show()

+-------+------------------+------------------+------------------+--------------------+-----------------+
|summary|      CarrierDelay|      WeatherDelay|          NASDelay|       SecurityDelay|LateAircraftDelay|
+-------+------------------+------------------+------------------+--------------------+-----------------+
|  count|          14595137|          14595137|          14595137|            14595137|         14595137|
|   mean|3.6357129090326454|0.7258234026854287| 3.686944562425142|0.027288335834052123|4.813298977597812|
| stddev|19.870826313275593| 9.085721457986146|15.910849753166524|  1.1833416733105868|20.59818066978041|
|    min|                 0|                 0|                 0|                   0|                0|
|    25%|                 0|                 0|                 0|                   0|                0|
|    50%|                 0|                 0|                 0|                   0|                0|
|    75%|                 0|                 0

Agrupando o somatório de tempos de atrasos por mês.

In [36]:
df_novo \
  .filter(df_novo.Cancelled == 0) \
  .groupby("Month") \
  .sum("CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay") \
  .show()

+-----+-----------------+-----------------+-------------+------------------+----------------------+
|Month|sum(CarrierDelay)|sum(WeatherDelay)|sum(NASDelay)|sum(SecurityDelay)|sum(LateAircraftDelay)|
+-----+-----------------+-----------------+-------------+------------------+----------------------+
|    7|          5607205|          1249421|      5094175|             34446|               7732828|
|    6|          5481092|          1411683|      5576463|             32409|               7761588|
|    9|          3586684|           652816|      3812771|             24657|               4245854|
|    5|          3646186|           704800|      3991750|             19245|               4890847|
|    1|          3916306|           855848|      4040153|             20180|               4876846|
|   10|          4037637|           727309|      4866278|             22987|               5348708|
|    3|          4444808|           738196|      4393814|             32017|               5979005|


Descobrindo o maior caso.

In [37]:
from pyspark.sql.functions import greatest

df_novo \
  .filter(df_novo.Cancelled == 0) \
  .groupby("Month") \
  .sum("CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay") \
  .withColumn('max_by_rows', greatest('sum(CarrierDelay)', 'sum(WeatherDelay)', 'sum(NASDelay)', 'sum(SecurityDelay)', 'sum(LateAircraftDelay)')) \
  .sort("Month") \
  .show()

+-----+-----------------+-----------------+-------------+------------------+----------------------+-----------+
|Month|sum(CarrierDelay)|sum(WeatherDelay)|sum(NASDelay)|sum(SecurityDelay)|sum(LateAircraftDelay)|max_by_rows|
+-----+-----------------+-----------------+-------------+------------------+----------------------+-----------+
|    1|          3916306|           855848|      4040153|             20180|               4876846|    4876846|
|    2|          4276567|           980141|      4012555|             36484|               5507852|    5507852|
|    3|          4444808|           738196|      4393814|             32017|               5979005|    5979005|
|    4|          3766487|           591164|      3946375|             37470|               5073410|    5073410|
|    5|          3646186|           704800|      3991750|             19245|               4890847|    4890847|
|    6|          5481092|          1411683|      5576463|             32409|               7761588|    7

Escrevendo na coluna 'MAX' o nome do tipo de Delay com maior atraso.

In [39]:
import pyspark.sql.functions as psf

columns = ['sum(CarrierDelay)', 'sum(WeatherDelay)', 'sum(NASDelay)', 'sum(SecurityDelay)', 'sum(LateAircraftDelay)']

# Pesquisando, encontramos a forma abaixo de descobrir o nome da coluna de maior valor
cond = "psf.when" + ".when".join(["(psf.col('" + c + "') == psf.col('max_by_rows'), psf.lit('" + c + "'))" for c in columns])

df_novo \
  .filter(df_novo.Cancelled == 0) \
  .groupby("Month") \
  .sum("CarrierDelay", "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay") \
  .sort("Month") \
  .withColumn("max_by_rows", psf.greatest(*columns))\
  .withColumn("MAX", eval(cond))\
  .show()


+-----+-----------------+-----------------+-------------+------------------+----------------------+-----------+--------------------+
|Month|sum(CarrierDelay)|sum(WeatherDelay)|sum(NASDelay)|sum(SecurityDelay)|sum(LateAircraftDelay)|max_by_rows|                 MAX|
+-----+-----------------+-----------------+-------------+------------------+----------------------+-----------+--------------------+
|    1|          3916306|           855848|      4040153|             20180|               4876846|    4876846|sum(LateAircraftD...|
|    2|          4276567|           980141|      4012555|             36484|               5507852|    5507852|sum(LateAircraftD...|
|    3|          4444808|           738196|      4393814|             32017|               5979005|    5979005|sum(LateAircraftD...|
|    4|          3766487|           591164|      3946375|             37470|               5073410|    5073410|sum(LateAircraftD...|
|    5|          3646186|           704800|      3991750|            